In [ ]:
import torch
from torch import Tensor
from train import train_gfn_sr

### Benchmark Nguyen sin ruido

In [ ]:
def h1(X: Tensor) -> Tensor:
    return X[:, 0].pow(3) + X[:, 0].pow(2) + X[:, 0]

def h2(X: Tensor) -> Tensor:
    return X[:, 0].pow(4) + X[:, 0].pow(3) + X[:, 0].pow(2) + X[:, 0]

def h3(X: Tensor) -> Tensor:
    return (
        X[:, 0].pow(5) + X[:, 0].pow(4) +
        X[:, 0].pow(3) + X[:, 0].pow(2) + X[:, 0]
    )

def h4(X: Tensor) -> Tensor:
    return (
        X[:, 0].pow(6) + X[:, 0].pow(5) +
        X[:, 0].pow(4) + X[:, 0].pow(3) +
        X[:, 0].pow(2) + X[:, 0]
    )

def h5(X: Tensor) -> Tensor:
    return X[:, 0].pow(2).sin() * X[:, 0].cos() - 1

def h6(X: Tensor) -> Tensor:
    return X[:, 0].sin() + (X[:, 0] + X[:, 0].pow(2)).sin()

def h7(X: Tensor) -> Tensor:
    return (X[:, 0] + 1).log() + (X[:, 0].pow(2) + 1).log()

def h8(X: Tensor) -> Tensor:
    return X[:, 0].sqrt()

def h9(X: Tensor, Y: Tensor) -> Tensor:
     return X[:, 0].sin() + Y[:, 0].pow(2).sin()

def h10(X: Tensor, Y: Tensor) -> Tensor:
    return 2 * X[:, 0].sin() * Y[:, 0].cos()

def h11(X: Tensor, Y: Tensor) -> Tensor:
    return X[:, 0].pow(Y[:, 0])

def h12(X: Tensor, Y: Tensor) -> Tensor:
    return (
        X[:, 0].pow(4) - X[:, 0].pow(3) +
        0.5 * Y[:, 0].pow(2) - Y[:, 0]
    )


#### Intervalos considerados

In [ ]:
FUNC_PARAMS = {
    h1: {"interval": (-1, 1), "n_variables": 1, "expr": "x**3 + x**2 + x"},
    h2: {"interval": (-1, 1), "n_variables": 1, "expr": "x**4 + x**3 + x**2 + x"},
    h3: {"interval": (-1, 1), "n_variables": 1, "expr": "x**5 + x**4 + x**3 + x**2 + x"},
    h4: {"interval": (-1, 1), "n_variables": 1, "expr": "x**6 + x**5 + x**4 + x**3 + x**2 + x"},
    h5: {"interval": (-1, 1), "n_variables": 1, "expr": "sin(x**2)cos(x) - 1"},
    h6: {"interval": (-1, 1), "n_variables": 1, "expr": "sin(x) + sin(x + x**2)"},
    h7: {"interval": (0, 2), "n_variables": 1, "expr": "log(x + 1) + log(x**2 + 1)"},
    h8: {"interval": (0, 4), "n_variables": 1, "expr": "sqrt(x)"},
    h9: {"interval": (-1, 1), "n_variables": 2, "expr": "sin(x) + sin(y**2)"},
    h10: {"interval": (-1, 1), "n_variables": 2, "expr": "2sin(x)cos(y)"},
    # h11: {"interval": (1, 3), "n_variables": 2, "expr": "x**y"},
    # h12: {"interval": (0, 2), "n_variables": 2, "expr": "x**4 - x**3 + 0.5 * y**2 - y"},
}

In [ ]:
batch_size = 64
num_epochs = 5000

In [ ]:
for i, (function, params) in enumerate(FUNC_PARAMS.items()):
    print('-'*50)
    print(f"FUNCTION {params['expr']}")
    low_bound, upp_bound = params["interval"]
    # Traslación lineal al intervalo correspondiente
    X = torch.empty(200, params["n_variables"]).uniform_(0, 1) * (upp_bound - low_bound) + low_bound
    y = function(*X.split(1, dim=1))
    model, env, errs, avg_mses, top_mses = train_gfn_sr(X, y, batch_size, num_epochs, show_plot=True, use_gpu=True)
    torch.save(model, f"h_{i+1}_no_noise.pt")